# Implementation of A\*


This article is a companion guide to my [introduction to A\*](https://www.redblobgames.com/pathfinding/a-star/introduction.html). I show how to implement Breadth-First Search, Dijkstra’s Algorithm, Greedy Best-First Search, and A*. I focused on keeping things simple.

Graph search is a family of related algorithms. There are lots of variants of the algorithms, and lots of variants in implementation. Treat the code on this page as a starting point, not as a final version of the algorithm that works for all situations.

## 1. Python Implementation

I explain most of the code below. There are a few extra bits that you can find in [implementation.py](https://www.redblobgames.com/pathfinding/a-star/implementation.py). These use Python 3 so if you use Python 2, you will need to change the super() call and the print function to the Python 2 equivalents.

### 1.1 Breadth First Search

Let’s implement Breadth First Search in Python. The main article shows the Python code for the search algorithm, but we also need to define the graph it works on. These are the abstractions I’ll use:

* Graph: a data structure that can tell me the neighbors for each location (see this tutorial). A weighted graph can also tell me the cost of moving along an edge.
* Locations: a simple value (int, string, tuple, etc.) that labels locations in the graph
* Search: an algorithm that takes a graph, a starting location, and optionally a goal location, and calculates some useful information (visited, parent pointer, distance) for some or all locations
* Queue: a data structure used by the search algorithm to decide the order in which to process the locations.

In the main article, I focused on search. On this page, I’ll fill in the rest of the details to make complete working programs. Let’s start with a graph:

```python
class SimpleGraph:
    def __init__(self):
        self.edges = {}
    
    def neighbors(self, id):
        return self.edges[id]
```

Yep, that’s all we need! You may be asking, where’s the Node object? The answer is: I rarely use a node object. I find it simpler to use integers, strings, or tuples as locations, and then use arrays or hash tables that use locations as an index.

Note that the edges are directed: we can have an edge from A to B without also having an edge from B to A. In game maps most edges are bidirectional but sometimes there are one-way doors or jumps off cliffs that are expressed as directed edges. Let’s make an example graph, where the locations are letters A-E.

<img src="source/Source_AStar_graph.png">

For each location I need a list of which locations it leads to:

```python
example_graph = SimpleGraph()
example_graph.edges = {
    'A': ['B'],
    'B': ['A', 'C', 'D'],
    'C': ['A'],
    'D': ['E', 'A'],
    'E': ['B']
}
```

Before we can use it with a search algorithm, we need to make a queue:

```python
import collections

class Queue:
    def __init__(self):
        self.elements = collections.deque()
    
    def empty(self):
        return len(self.elements) == 0
    
    def put(self, x):
        self.elements.append(x)
    
    def get(self):
        return self.elements.popleft()
```

This queue class is just a wrapper around the built-in collections.deque class. Feel free to use deque directly in your own code.

Let’s try the example graph with this queue and the breadth-first search algorithm code from the main article:

In [22]:
from collections import deque
import heapq

from graph import *

In [23]:
def breadth_first_search(graph, start):
    # store the nodes (state) in the frontier
    frontier = deque()
    frontier.append(start)
    
    # store the visited node (state)
    visited = {}
    visited[start] = True
    
    while frontier:
        curr_node = frontier.popleft()
        print("Visiting {}".format(curr_node))
        
        for next_node in graph[curr_node]:  # graph.neighbors(curr_node)
            if next_node not in visited:
                frontier.append(next_node)
                visited[next_node] = True
                

graph = {
    'A': ['B'],
    'B': ['A', 'C', 'D'],
    'C': ['A'],
    'D': ['E', 'A'],
    'E': ['B']
}
breadth_first_search(graph, start='A')

Visiting A
Visiting B
Visiting C
Visiting D
Visiting E


Grids can be expressed as graphs too. I’ll now define a new graph called SquareGrid, with locations tuples (int, int). Instead of storing the edges explicitly, I’ll calculate them in the neighbors function.

In [24]:
g = SquareGrid(30, 15)
g.walls = DIAGRAM1_WALLS # long list, [(21, 0), (21, 2), ...]
draw_grid(g)

. . . . . . . . . . . . . . . . . . . . . ####. . . . . . . 
. . . . . . . . . . . . . . . . . . . . . ####. . . . . . . 
. . . . . . . . . . . . . . . . . . . . . ####. . . . . . . 
. . . ####. . . . . . . . . . . . . . . . ####. . . . . . . 
. . . ####. . . . . . . . ####. . . . . . ####. . . . . . . 
. . . ####. . . . . . . . ####. . . . . . ##########. . . . 
. . . ####. . . . . . . . ####. . . . . . ##########. . . . 
. . . ####. . . . . . . . ####. . . . . . . . . . . . . . . 
. . . ####. . . . . . . . ####. . . . . . . . . . . . . . . 
. . . ####. . . . . . . . ####. . . . . . . . . . . . . . . 
. . . ####. . . . . . . . ####. . . . . . . . . . . . . . . 
. . . ####. . . . . . . . ####. . . . . . . . . . . . . . . 
. . . . . . . . . . . . . ####. . . . . . . . . . . . . . . 
. . . . . . . . . . . . . ####. . . . . . . . . . . . . . . 
. . . . . . . . . . . . . ####. . . . . . . . . . . . . . . 


In order to reconstruct paths we need to store the location of where we came from, so I’ve renamed visited (True/False) to came_from (location):

In [25]:
def breadth_first_search_2(graph, start):
    # store the nodes (state) in the frontier
    frontier = deque()
    frontier.append(start)
    
    # store the visited node (state)
    came_from = {}
    came_from[start] = None
    
    while frontier:
        curr_node = frontier.popleft()

        for next_node in graph.neighbors(curr_node):  # graph.neighbors(curr_node)
            if next_node not in came_from:
                frontier.append(next_node)
                came_from[next_node] = curr_node
                
    return came_from

g = SquareGrid(30, 15)
g.walls = DIAGRAM1_WALLS

parents = breadth_first_search_2(g, (8, 7))
draw_grid(g, width=2, point_to=parents, start=(8, 7))

> > > > v v v v v v v v v v v v < < < < < ####v v v v v v v 
> > > > > v v v v v v v v v v < < < < < < ####v v v v v v v 
> > > > > v v v v v v v v v < < < < < < < ####> v v v v v v 
> > ^ ####v v v v v v v v < < < < < < < < ####> > v v v v v 
> ^ ^ ####> v v v v v v < ####^ < < < < < ####> > > v v v v 
^ ^ ^ ####> > v v v v < < ####^ ^ < < < < ##########v v v < 
^ ^ ^ ####> > > v v < < < ####^ ^ ^ < < < ##########v v < < 
^ ^ ^ ####> > > A < < < < ####^ ^ ^ ^ < < < < < < < < < < < 
v v v ####> > ^ ^ ^ < < < ####^ ^ ^ ^ ^ < < < < < < < < < < 
v v v ####> ^ ^ ^ ^ ^ < < ####^ ^ ^ ^ ^ ^ < < < < < < < < < 
v v v ####^ ^ ^ ^ ^ ^ ^ < ####^ ^ ^ ^ ^ ^ ^ < < < < < < < < 
> v v ####^ ^ ^ ^ ^ ^ ^ ^ ####^ ^ ^ ^ ^ ^ ^ ^ < < < < < < < 
> > > > > ^ ^ ^ ^ ^ ^ ^ ^ ####^ ^ ^ ^ ^ ^ ^ ^ ^ < < < < < < 
> > > > ^ ^ ^ ^ ^ ^ ^ ^ ^ ####^ ^ ^ ^ ^ ^ ^ ^ ^ ^ < < < < < 
> > > ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ####^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ < < < < 


### 1.2 Early Exit

Following the code from the main article, all we need to do is add an if statement in the main loop. This test is optional for Breadth First Search or Dijkstra’s Algorithm and effectively required for Greedy Best-First Search and A\*:

In [26]:
def breadth_first_search_3(graph, start, goal):
    frontier = deque()
    frontier.append(start)
    came_from = {}
    came_from[start] = None
    
    while frontier:
        curr_node = frontier.popleft()
        
        if curr_node == goal:
            break
        
        for next_node in graph.neighbors(curr_node):
            if next_node not in came_from:
                frontier.append(next_node)
                came_from[next_node] = curr_node
    
    return came_from

g = SquareGrid(30, 15)
g.walls = DIAGRAM1_WALLS

parents = breadth_first_search_3(g, (8, 7), (17, 2))
draw_grid(g, width=2, point_to=parents, start=(8, 7), goal=(17, 2))

. > > > v v v v v v v v v v v v < . . . . ####. . . . . . . 
> > > > > v v v v v v v v v v < < < . . . ####. . . . . . . 
> > > > > v v v v v v v v v < < < Z . . . ####. . . . . . . 
> > ^ ####v v v v v v v v < < < < < < . . ####. . . . . . . 
. ^ ^ ####> v v v v v v < ####^ < < . . . ####. . . . . . . 
. . ^ ####> > v v v v < < ####^ ^ . . . . ##########. . . . 
. . . ####> > > v v < < < ####^ . . . . . ##########. . . . 
. . . ####> > > A < < < < ####. . . . . . . . . . . . . . . 
. . . ####> > ^ ^ ^ < < < ####. . . . . . . . . . . . . . . 
. . v ####> ^ ^ ^ ^ ^ < < ####. . . . . . . . . . . . . . . 
. v v ####^ ^ ^ ^ ^ ^ ^ < ####. . . . . . . . . . . . . . . 
> v v ####^ ^ ^ ^ ^ ^ ^ ^ ####. . . . . . . . . . . . . . . 
> > > > > ^ ^ ^ ^ ^ ^ ^ ^ ####. . . . . . . . . . . . . . . 
> > > > ^ ^ ^ ^ ^ ^ ^ ^ ^ ####. . . . . . . . . . . . . . . 
. > > ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ####. . . . . . . . . . . . . . . 


### 1.3 Dijkstra’s Algorithm

This is what adds complexity to graph search, because we’re going to start processing locations in a better order than “first in, first out”. What do we need to change?

1. The graph needs to know cost of movement.
2. The queue needs to return nodes in a different order.
3. The search needs to keep track of these costs from the graph and give them to the queue.

#### 1.3.1 Graph with weights

A regular graph tells me the neighbors of each node. A weighted graph also tells me the cost of moving along each edge. I’m going to add a cost(from_node, to_node) function that tells us the cost of moving from location from_node to its neighbor to_node. In this forest map I chose to make movement depend only on to_node, but there are other types of movement that use both nodes. An alternate implementation would be to merge this into the neighbors function.

```python
class GridWithWeights(SquareGrid):
    def __init__(self, width, height):
        super().__init__(width, height)
        self.weights = {}
    
    def cost(self, from_node, to_node):
        return self.weights.get(to_node, 1)
```

#### 1.3.2 Queue with priorities

A priority queue associates with each item a number called a “priority”. When returning an item, it picks the one with the lowest number.

* insert: Add item to queue
* remove: Remove item with the lowest number
* reprioritize: (optional) Change an existing item’s priority to a lower number

Here’s a reasonably fast priority queue that uses binary heaps, but does not support reprioritize. To get the right ordering, we’ll use tuples (priority, item). When an element is inserted that is already in the queue, we’ll have a duplicate; I’ll explain why that’s ok in the Optimization section.


```python
import heapq

class PriorityQueue:
    def __init__(self):
        self.elements = []
    
    def empty(self):
        return len(self.elements) == 0
    
    def put(self, item, priority):
        heapq.heappush(self.elements, (priority, item))
    
    def get(self):
        return heapq.heappop(self.elements)[1]
```

#### 1.3.3 Search

Here’s a tricky bit about the implementation: once we add movement costs it’s possible to visit a location again, with a better cost_so_far. That means the line if next not in came_from won’t work. Instead, have to check if the cost has gone down since the last time we visited. (In the original version of the article I wasn’t checking this, but my code worked anyway; [I wrote some notes about that bug](https://www.redblobgames.com/pathfinding/posts/reprioritize.html).)

This forest map is from [the main page](https://www.redblobgames.com/pathfinding/a-star/introduction.html#dijkstra).

In [27]:
def dijkstra_search(graph, start, goal):
    
    frontier = []
    heapq.heappush(frontier, (0 ,start)) # node, cost
    
    came_from = {}
    came_from[start] = None
    
    cost_so_far = {}    
    cost_so_far[start] = 0
    
    while frontier:
        curr_cost, curr_node = heapq.heappop(frontier)
        
        if curr_node == goal:
            break
        
        for next_node in graph.neighbors(curr_node):
            new_cost = cost_so_far[curr_node] + graph.cost(curr_node, next_node)
            if next_node not in cost_so_far or new_cost < cost_so_far[next_node]:
                cost_so_far[next_node] = new_cost
                priority = new_cost
                heapq.heappush(frontier, (priority, next_node))
                came_from[next_node] = curr_node
    
    return came_from, cost_so_far

In [29]:
def reconstruct_path(came_from, start, goal):
    current = goal
    path = []
    while current != start:
        path.append(current)
        current = came_from[current]
    path.append(start) # optional
    path.reverse() # optional
    return path

Although paths are best thought of as a sequence of edges, it’s convenient to store them as a sequence of nodes. To build the path, start at the end and follow the came_from map, which points to the previous node. When we reach start, we’re done. It is the backwards path, so call reverse() at the end of reconstruct_path if you need it to be stored forwards. Sometimes it’s actually more convenient to store it backwards. Sometimes it’s useful to also store the start node in the list.

Let’s try it out:

In [30]:
came_from, cost_so_far = dijkstra_search(diagram4, (1, 4), (7, 8))
draw_grid(diagram4, width=3, point_to=came_from, start=(1, 4), goal=(7, 8))
print()
draw_grid(diagram4, width=3, number=cost_so_far, start=(1, 4), goal=(7, 8))
print()
draw_grid(diagram4, width=3, path=reconstruct_path(came_from, start=(1, 4), goal=(7, 8)))

v  v  <  <  <  <  <  <  <  <  
v  v  <  <  <  ^  ^  <  <  <  
v  v  <  <  <  <  ^  ^  <  <  
v  v  <  <  <  <  <  ^  ^  .  
>  A  <  <  <  <  .  .  .  .  
^  ^  <  <  <  <  .  .  .  .  
^  ^  <  <  <  <  <  .  .  .  
^  #########^  <  v  .  .  .  
^  #########v  v  v  Z  .  .  
^  <  <  <  <  <  <  <  <  .  

5  4  5  6  7  8  9  10 11 12 
4  3  4  5  10 13 10 11 12 13 
3  2  3  4  9  14 15 12 13 14 
2  1  2  3  8  13 18 17 14 .  
1  A  1  6  11 16 .  .  .  .  
2  1  2  7  12 17 .  .  .  .  
3  2  3  4  9  14 19 .  .  .  
4  #########14 19 18 .  .  .  
5  #########15 16 13 Z  .  .  
6  7  8  9  10 11 12 13 14 .  

.  .  .  .  .  .  .  .  .  .  
.  .  .  .  .  .  .  .  .  .  
.  .  .  .  .  .  .  .  .  .  
.  .  .  .  .  .  .  .  .  .  
@  @  .  .  .  .  .  .  .  .  
@  .  .  .  .  .  .  .  .  .  
@  .  .  .  .  .  .  .  .  .  
@  #########.  .  .  .  .  .  
@  #########.  .  @  @  .  .  
@  @  @  @  @  @  @  .  .  .  


The line if next not in cost_so_far or new_cost < cost_so_far[next] could be simplified to if new_cost < cost_so_far.get(next, Infinity) but I didn’t want to explain Python’s get() in the main article so I left it as is. Another approach would be to use collections.defaultdict defaulting to infinity.

### 1.4 A\* Search

Both Greedy Best-First Search and A\* use a heuristic function. The only difference is that A\* uses both the heuristic and the ordering from Dijkstra’s Algorithm. I’m going to show A\* here.

In [34]:
def heuristic(a, b):
    """
    Manhattan distance:
    The sum of the horizontal and vertical distances between points on a grid
    """
    (x1, y1) = a
    (x2, y2) = b
    return abs(x1 - x2) + abs(y1 - y2)

def a_star_search(graph, start, goal):
    frontier = []
    heapq.heappush(frontier, (0 ,start)) # node, cost
    
    came_from = {}
    came_from[start] = None
    
    cost_so_far = {}    
    cost_so_far[start] = 0
    
    while frontier:
        curr_cost, curr_node = heapq.heappop(frontier)
        
        if curr_node == goal:
            break
        
        for next_node in graph.neighbors(curr_node):
            new_cost = cost_so_far[curr_node] + graph.cost(curr_node, next_node)
            if next_node not in cost_so_far or new_cost < cost_so_far[next_node]:
                cost_so_far[next_node] = new_cost
                priority = new_cost + heuristic(goal, next_node)
                heapq.heappush(frontier, (priority, next_node))
                came_from[next_node] = curr_node
    
    return came_from, cost_so_far

Let’s try it out:

In [35]:
start, goal = (1, 4), (7, 8)
came_from, cost_so_far = a_star_search(diagram4, start, goal)
draw_grid(diagram4, width=3, point_to=came_from, start=start, goal=goal)
print()
draw_grid(diagram4, width=3, number=cost_so_far, start=start, goal=goal)
print()

.  .  .  .  .  .  .  .  .  .  
.  v  v  v  .  .  .  .  .  .  
v  v  v  v  <  .  .  .  .  .  
v  v  v  <  <  .  .  .  .  .  
>  A  <  <  <  .  .  .  .  .  
>  ^  <  <  <  .  .  .  .  .  
>  ^  <  <  <  <  .  .  .  .  
^  #########^  .  v  .  .  .  
^  #########v  v  v  Z  .  .  
^  <  <  <  <  <  <  <  .  .  

.  .  .  .  .  .  .  .  .  .  
.  3  4  5  .  .  .  .  .  .  
3  2  3  4  9  .  .  .  .  .  
2  1  2  3  8  .  .  .  .  .  
1  A  1  6  11 .  .  .  .  .  
2  1  2  7  12 .  .  .  .  .  
3  2  3  4  9  14 .  .  .  .  
4  #########14 .  18 .  .  .  
5  #########15 16 13 Z  .  .  
6  7  8  9  10 11 12 13 .  .  



#### 1.4.1 Straighter paths
If you implement this code in your own project you might find that some of the paths aren’t as “straight” as you’d like. This is normal. When using grids, especially grids where every step has the same movement cost, you end up with ties: many paths have exactly the same cost. A* ends up picking one of the many short paths, and very often it doesn’t look good to you. The quick hack is to break the ties, but it’s not entirely satisfactory. The better approach is to change the map representation, which makes A* a lot faster, and also produces straighter, better looking paths. However, that only works for mostly-static maps where every step has the same movement cost. For the demos on my page, I’m using a quick hack, but it only works with my slow priority queue. If you switch to a faster priority queue you’ll need a different quick hack.

## 2. Algorithm Changes

The version of Dijkstra’s Algorithm and A\* on my pages is slightly different from what you’ll see in an algorithms or AI textbook.

The pure version of Dijkstra’s Algorithm starts the priority queue with all nodes, and does not have early exit. It uses a “decrease-key” operation in the queue. It’s fine in theory. But in practice…

1. By starting the priority with only the start node, we can keep it small, which makes it faster and use less memory.
2. With early exit, we almost never need to insert all the nodes into the queue, and we can return the path as soon as it’s found.
3. By not putting all nodes into the queue at the start, most of the time we can use a cheap insert operation instead of the more expensive decrease-key operation.
4. By not putting all nodes into the queue at the start, we can handle situations where we do not even know all the nodes, or where the number of nodes is infinite.  

This variant is sometimes called “Uniform Cost Search”. See Wikipedia to see the pseudocode, or read [Felner’s paper](https://www.aaai.org/ocs/index.php/SOCS/SOCS11/paper/viewFile/4017/4357) to see justifications for these changes.

There are three further differences between my version and what you might find elsewhere. These apply to both Dijkstra’s Algorithm and A\*:

1. I eliminate the check for a node being in the frontier with a higher cost. By not checking, I end up with duplicate elements in the frontier. The algorithm still works. It will revisit some locations more than necessary (but rarely, in my experience). The code is simpler and it allows me to use a simpler and faster priority queue that does not support the decrease-key operation. The paper [“Priority Queues and Dijkstra’s Algorithm”](http://www.cs.sunysb.edu/~rezaul/papers/TR-07-54.pdf) suggests that this approach is faster in practice.
2. Instead of storing both a “closed set” and an “open set”, I have a visited flag that tells me whether it’s in either of those sets. This simplifies the code further.
3. I don’t need to store a separate open or closed set explicitly because the set is implicit in the keys of the came_from and cost_so_far tables. Since we always want one of those two tables, there’s no need to store the open/closed sets separately.
4. I use hash tables instead of arrays of node objects. This eliminates the rather expensive initialize step that many other implementations have. For large game maps, the initialization of those arrays is often slower than the rest of A\*.

If you have more suggestions for simplifications that preserve performance, please let me know!